# **Collecte de données**

In [23]:
import os, sys
from PIL import Image
from PIL.ExifTags import TAGS
chemin_image='./cattt'
for fichier in os.listdir(chemin_image):
    if fichier.endswith('.jpg') :
        filepath = os.path.join(chemin_image, fichier)
        try:
            imgfile = Image.open(filepath)
            # Recupérer les métadonnées EXIF
            exif_data = imgfile._getexif()
            if exif_data is None:
                 continue
            if exif_data:
                print("Métadonnées pour l'image {filename}:")
            for tag,value in exif_data.items():
                if tag in TAGS:
                    print(f"{TAGS[tag]}: {value}")
                else:
                    print(f"Aucune métadonné EXIF disponible pour l'image {imgfile}.")
        except (OSError, AttributeError)as e:
                print(f"Impossible de traiter l'image {imgfile}.{e}")



Nous n'avons aucune image avec un exif donc nous passons par uen autre méthode

In [1]:
import os, sys
from PIL import Image
from PIL.ExifTags import TAGS

chemin_image='./cattt'
chemin_json='./metadonnees_images.json'

metadatalist=[]

for fichier in os.listdir(chemin_image):
    if fichier.endswith('.jpg') :
        filepath = os.path.join(chemin_image, fichier)
        try:
            img = Image.open(filepath)
            metadata={
                 'file_name': fichier,
                 'size': img.size,
                 'format': img.format,
                 'mode': img.mode,
                 'exif': None
            }

            metadatalist.append(metadata)

        except (OSError, AttributeError)as e:
                print(f"Impossible de traiter l'image {imgfile}.{e}")

In [9]:
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
import os
from sklearn.cluster import KMeans

chemin_image='./cattt'

for fichier in os.listdir(chemin_image):
    if fichier.endswith('.jpg') :
        filepath = os.path.join(chemin_image, fichier)
        imgfile = Image.open(fichier)
        numarray = numpy.array(imgfile.getdata(), numpy.uint8)
        clusters = KMeans(n_clusters=4, n_init=2)
        clusters.fit(numarray)
        npbins = numpy.arange(0, 5)
        histogram = numpy.histogram(clusters.labels_, bins=npbins)
        labels = numpy.unique(clusters.labels_)
        barlist = plot.bar(labels, histogram[0])
        for i in range(4):
            barlist[i].set_color(
                "#%02x%02x%02x"
                % (
                    math.ceil(clusters.cluster_centers_[i][0]),
                    math.ceil(clusters.cluster_centers_[i][1]),
                    math.ceil(clusters.cluster_centers_[i][2]),
                )
            )
plot.show()



FileNotFoundError: [Errno 2] No such file or directory: '00000016_022.jpg'

Visualisation des données 

In [10]:
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
import os
from sklearn.cluster import KMeans

for fichier in os.listdir(chemin_image):
    if fichier.endswith('.jpg') :
        filepath = os.path.join(chemin_image, fichier)
        imgfile = Image.open(fichier)
        histogram = imgfile.histogram()
        red = histogram[0:255]
        green = histogram[256:511]
        blue = histogram[512:767]
        x = range(255)
        y = []
        for i in x:
            y.append((red[i], green[i], blue[i]))
        figure, axes = plot.subplots()
        axes.set_prop_cycle("color", ["red", "green", "blue"])
        plot.plot(x, y)
        plot.show()


FileNotFoundError: [Errno 2] No such file or directory: '00000016_022.jpg'